# Transforming data
- 여기에서는 머신러닝 알고리즘이 학습할 수 있는 형태로 데이터를 변환(transform)하는 과정에 대해 다룹니다.
- 주로 `scikit-learn` 패키지 내에 있는 `preprocessing` 모듈을 통해 데이터를 변환하는 방법에 대해 알아봅시다.
- 데이터 변환에 도움을 주는 함수를 익히는 것도 중요하지만, **어떤 과정을 통해 데이터를 분석 가능한 형태로 변환**하는지 파악하는 것이 제일 중요합니다!

In [1]:
import pandas as pd
import numpy as np

## 1. Rescale the data 
- Also known as **normalization**
- 각 변수의 값을 내가 원하는 range로 변경 (예. 0과 1사이)
- 머신러닝의 핵심인 최적화 알고리즘을 적용하는 데에 매우 유용함 (예. Gradient descent)
- 포인트 간의 거리를 기반으로 하는 알고리즘에서는 핵심 (예. k-means clustering, k-nearest neighbors)

### Import example dataset 

In [2]:
variables = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
data = pd.read_csv('data/pima-indians-diabetes.data', names = variables)
data.head(10)

,preg,plas,pres,skin,test,mass,pedi,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
5,5,116,74,0,0,25.6,0.201,30,0
6,3,78,50,32,88,31.0,0.248,26,1
7,10,115,0,0,0,35.3,0.134,29,0
8,2,197,70,45,543,30.5,0.158,53,1
9,8,125,96,0,0,0.0,0.232,54,1


In [3]:
data.shape

(768, 9)

In [4]:
num_rows = data.shape[0]
num_cols = data.shape[1]
print(num_rows)
print(num_cols)

768
9


In [5]:
data.dtypes

preg       int64
plas       int64
pres       int64
skin       int64
test       int64
mass     float64
pedi     float64
age        int64
class      int64
dtype: object

### DataFrame to Array
- 본 방법에서는 `scikit-learn` 내 `MinMaxScaler`라는 함수를 사용할 예정
- 이 함수는 numpy array에서만 적용 가능함

In [5]:
array = data.values
array

array([[   6.   ,  148.   ,   72.   , ...,    0.627,   50.   ,    1.   ],
       [   1.   ,   85.   ,   66.   , ...,    0.351,   31.   ,    0.   ],
       [   8.   ,  183.   ,   64.   , ...,    0.672,   32.   ,    1.   ],
       ..., 
       [   5.   ,  121.   ,   72.   , ...,    0.245,   30.   ,    0.   ],
       [   1.   ,  126.   ,   60.   , ...,    0.349,   47.   ,    1.   ],
       [   1.   ,   93.   ,   70.   , ...,    0.315,   23.   ,    0.   ]])

- 데이터에서 클래스는 0과 1사이로 스케일링이 될 필요가 없다.
- 따라서 스케일이 필요한 입력변수들을 X, 클래스를 Y로 분리를 한 후 X에만 스케일링

In [6]:
# -1은 마지막 인덱스를 의미함
# array[:, 0:-1] 은 모든 행, 첫 번째 열부터 마지막 열 직전까지의 열을 따로 분리한다는 뜻
X = array[:,0:-1]

In [7]:
Y = array[:,-1]

In [8]:
X.shape

(768, 8)

In [9]:
Y.shape

(768,)

### Using `sklearn.preprocessing.MinMaxScaler`
- 참고: http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html

In [10]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
rescaledX = scaler.fit_transform(X)
np.set_printoptions(precision=3)
print(X[0:5,:])
print(rescaledX[0:5,:])

[[  6.000e+00   1.480e+02   7.200e+01   3.500e+01   0.000e+00   3.360e+01
    6.270e-01   5.000e+01]
 [  1.000e+00   8.500e+01   6.600e+01   2.900e+01   0.000e+00   2.660e+01
    3.510e-01   3.100e+01]
 [  8.000e+00   1.830e+02   6.400e+01   0.000e+00   0.000e+00   2.330e+01
    6.720e-01   3.200e+01]
 [  1.000e+00   8.900e+01   6.600e+01   2.300e+01   9.400e+01   2.810e+01
    1.670e-01   2.100e+01]
 [  0.000e+00   1.370e+02   4.000e+01   3.500e+01   1.680e+02   4.310e+01
    2.288e+00   3.300e+01]]
[[ 0.353  0.744  0.59   0.354  0.     0.501  0.234  0.483]
 [ 0.059  0.427  0.541  0.293  0.     0.396  0.117  0.167]
 [ 0.471  0.92   0.525  0.     0.     0.347  0.254  0.183]
 [ 0.059  0.447  0.541  0.232  0.111  0.419  0.038  0.   ]
 [ 0.     0.688  0.328  0.354  0.199  0.642  0.944  0.2  ]]


## 2. Standardize the data 
- 각 변수의 값을 standard Gaussian distribution (with a mean of 0 and a standard deviation of 1) 를 따르도록 데이터를 바꾸는 방법
- 입력변수가 지나치게 왜도(skewness)가 크지 않을 때에 특히 효과적
- 몇 가지 머신러닝 알고리즘 중 입력변수의 정규성을 가정하거나 rescale이 효과적인 것에 적합함 (예. Linear regression, Logistic regression, Linear discriminant analysis 등)

### Using `sklearn.preprocessing.StandardScaler`
- 참고: http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

In [12]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
rescaledX = scaler.fit_transform(X)
np.set_printoptions(precision=3)
print(rescaledX[0:5, :])

[[ 0.64   0.848  0.15   0.907 -0.693  0.204  0.468  1.426]
 [-0.845 -1.123 -0.161  0.531 -0.693 -0.684 -0.365 -0.191]
 [ 1.234  1.944 -0.264 -1.288 -0.693 -1.103  0.604 -0.106]
 [-0.845 -0.998 -0.161  0.155  0.123 -0.494 -0.921 -1.042]
 [-1.142  0.504 -1.505  0.907  0.766  1.41   5.485 -0.02 ]]


## 3. Normalize each point 
- Normalizing in scikit-learn refers to rescaling each point (row) to have a length of 1 (called a unit norm or a vector with the length of 1 in linear algebra).
- Useful for sparse datasets (lots of zeros) with attributes of varying scales when using algorithms that weight input values such as neural networks and algorithms that use distance measures such as K-Nearest Neighbors.

### Using `sklearn.preprocessing.Normalizer`
- 참고: http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Normalizer.html

In [12]:
from sklearn.preprocessing import Normalizer
# scaler = Normalizer().fit(X)
# normalizedX = scaler.transform(X)
normalizedX = scaler.fit_transform(X)
np.set_printoptions(precision=3)
print(normalizedX[0:5,:])

[[ 0.353  0.744  0.59   0.354  0.     0.501  0.234  0.483]
 [ 0.059  0.427  0.541  0.293  0.     0.396  0.117  0.167]
 [ 0.471  0.92   0.525  0.     0.     0.347  0.254  0.183]
 [ 0.059  0.447  0.541  0.232  0.111  0.419  0.038  0.   ]
 [ 0.     0.688  0.328  0.354  0.199  0.642  0.944  0.2  ]]


## 4. Binarize features
- Thresholding numerical features to get boolean values
- 예제: 텍스트 분석에서 확률적 추론을 간단하게 만들기 위한 목적

In [14]:
from sklearn.preprocessing import Binarizer
X = [[ 1., -1.,  2.], [ 2.,  0.,  0.], [ 0.,  1., -1.]]
binarizer = Binarizer(threshold = 1.1)
binarizedX = binarizer.transform(X)
print(np.array(X))
print("==============")
print(binarizedX)

[[ 1. -1.  2.]
 [ 2.  0.  0.]
 [ 0.  1. -1.]]
[[ 0.  0.  1.]
 [ 1.  0.  0.]
 [ 0.  0.  0.]]


## 5. Encoding categorical variables
- Categorical variable: 값이 nominal인 변수
- 한 variable에 category가 총 C개가 존재하는 경우, 이를 C개의 binary dummy variables로 변환하여 수치형 데이터로 변환할 수 있다.
- 이를 **1 of C coding** 또는 **one hot encoding** 라고 한다.
- Using `sklearn.preprocessing.OneHotEncoder`
- Using `pandas.get_dummies`

### Using `sklearn.preprocessing.OneHotEncoder`
- 참고: http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html
- Category가 이미 숫자로 표현되어 있는 경우에 유리함. (예. 봄->1, 여름->2, 가을->3, 겨울->4)

In [13]:
from sklearn.preprocessing import OneHotEncoder
X = [[0,0,3],[1,1,0],[0,2,1],[1,0,2]]

In [14]:
encoder = OneHotEncoder()
encoder.fit(X)

OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=True)

In [15]:
encoder.transform([[0,1,3]]).toarray()

array([[ 1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.]])

### Using `pandas.get_dummies`
- 참고: http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html
- DataFrame에서 바로 binary dummies를 생성하므로 매우 직관적임.

In [16]:
data = {'temperature': [29, 34, 36, 32, 30],
       'humidity': ['mid', 'high', 'high', 'mid', 'low'],
       'weather': ['cloudy', 'sunny', 'rainy', 'cloudy', 'sunny']}
X = pd.DataFrame(data)

In [17]:
X

,humidity,temperature,weather
0,mid,29,cloudy
1,high,34,sunny
2,high,36,rainy
3,mid,32,cloudy
4,low,30,sunny


In [18]:
encodedX = pd.get_dummies(X)

In [19]:
encodedX

,temperature,humidity_high,humidity_low,humidity_mid,weather_cloudy,weather_rainy,weather_sunny
0,29,0,0,1,1,0,0
1,34,1,0,0,0,0,1
2,36,1,0,0,0,1,0
3,32,0,0,1,1,0,0
4,30,0,1,0,0,0,1


## 6. Missing value imputation
- Missing value(결측치)는 행렬 계산 기반의 머신러닝 알고리즘의 경우에는 반드시 처리를 해주어야 한다. (예. Linear regression, Logistic regression, Support vector machine, 등)

### Using `sklearn.preprocessing.Imputer`
- 참고: http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Imputer.html

In [20]:
from sklearn.preprocessing import Imputer
X = [[1,2],[np.nan,3],[7,6]]
imp = Imputer(missing_values = 'NaN', strategy = 'mean', axis = 0)
imp.fit(X)

Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)

In [21]:
X

[[1, 2], [nan, 3], [7, 6]]

In [22]:
test = [[np.nan,2],[6,np.nan],[7,6]]
print(test)

[[nan, 2], [6, nan], [7, 6]]


In [23]:
print(imp.transform(test))

[[ 4.     2.   ]
 [ 6.     3.667]
 [ 7.     6.   ]]


## 7. Custom transformers
- 사용자 정의의 함수를 이용하여 변수를 transform하는 방법

### Using `sklearn.preprocessing.FunctionTransformer`
- 참고: http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html

In [24]:
from sklearn.preprocessing import FunctionTransformer
transformer = FunctionTransformer(np.log1p) # numpy.log1p(x) calculates log(1 + x)
X = [[0, 1], [2, 3]]
transformer.transform(X)

array([[ 0.   ,  0.693],
       [ 1.099,  1.386]])

## 더 많은 정보는...
- `sklearn.preprocessing` 모듈에 대해 자세히 설명된 페이지: http://scikit-learn.org/stable/modules/preprocessing.html
- 데이터 분석을 위해 거쳐야 하는 전처리 과정을 잘 숙지한 경우, googling 및 stackoverflow에서 더욱 많은 정보를 확인할 수 있다.